In [1]:
#Regression
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
%matplotlib inline
import warnings
# warnings.filterwarnings('ignore')
from sklearn.model_selection import KFold, cross_val_score, train_test_split, TimeSeriesSplit
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, VotingRegressor, StackingRegressor, AdaBoostRegressor, HistGradientBoostingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, f1_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder,OrdinalEncoder, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.inspection import PartialDependenceDisplay
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [2]:
path = os.getcwd()
file_path = os.path.join(path,"file")
sub_path = os.path.join(path,"제출")
sub_num = 39
train = pd.read_csv(os.path.join(file_path,"train.csv"))
test = pd.read_csv(os.path.join(file_path,"test.csv"),index_col='timestamp', parse_dates=True)
ss = pd.read_csv(os.path.join(file_path,"sample_submission.csv"))

In [3]:
from pytimekr import pytimekr
import datetime

List_holiday = []
for i in range(5,1523,7):
    List_holiday.append((datetime.datetime(2019, 1, 1)+datetime.timedelta(i)).strftime("%Y-%m-%d"))
    
for i in range(2019,2024):
    List_holiday.extend(pytimekr.holidays(year=i)) #리스트 형태로 반환

for i in range(len(List_holiday)):
    List_holiday[i] = str(List_holiday[i])
np.sort(List_holiday)

array(['2019-01-01', '2019-01-06', '2019-01-13', '2019-01-20',
       '2019-01-27', '2019-02-03', '2019-02-04', '2019-02-05',
       '2019-02-06', '2019-02-10', '2019-02-17', '2019-02-24',
       '2019-03-01', '2019-03-03', '2019-03-10', '2019-03-17',
       '2019-03-24', '2019-03-31', '2019-04-07', '2019-04-14',
       '2019-04-21', '2019-04-28', '2019-05-05', '2019-05-05',
       '2019-05-12', '2019-05-12', '2019-05-19', '2019-05-26',
       '2019-06-02', '2019-06-06', '2019-06-09', '2019-06-16',
       '2019-06-23', '2019-06-30', '2019-07-07', '2019-07-14',
       '2019-07-21', '2019-07-28', '2019-08-04', '2019-08-11',
       '2019-08-15', '2019-08-18', '2019-08-25', '2019-09-01',
       '2019-09-08', '2019-09-12', '2019-09-13', '2019-09-14',
       '2019-09-15', '2019-09-22', '2019-09-29', '2019-10-03',
       '2019-10-06', '2019-10-09', '2019-10-13', '2019-10-20',
       '2019-10-27', '2019-11-03', '2019-11-10', '2019-11-17',
       '2019-11-24', '2019-12-01', '2019-12-08', '2019-

In [4]:
#쥬혁이님 코드 활용
darts_data = train.rename(columns={'timestamp':'ds', 'price(원/kg)':'y'})
darts_data = darts_data[['ID','ds','y']]
darts_data['ID'] = darts_data['ID'].str.replace(r'_\d{8}$','', regex=True)

In [5]:
darts_data

,ID,ds,y
0,TG_A_J,2019-01-01,0.0
1,TG_A_J,2019-01-02,0.0
2,TG_A_J,2019-01-03,1728.0
3,TG_A_J,2019-01-04,1408.0
4,TG_A_J,2019-01-05,1250.0
...,...,...,...
59392,RD_F_J,2023-02-27,468.0
59393,RD_F_J,2023-02-28,531.0
59394,RD_F_J,2023-03-01,574.0
59395,RD_F_J,2023-03-02,523.0


In [6]:
from darts.timeseries import TimeSeries
from darts.models import DLinearModel

random_seed = 1016
np.random.seed(random_seed)

#쥬혁이님 코드 활용
def ph_train(df):
    pred_list = []
    for code in darts_data['ID'].unique():
        d = darts_data[darts_data['ID'] == code].reset_index().drop(['ID'], axis=1).sort_values('ds')

        ts = TimeSeries.from_dataframe(d[['y']].astype('float32'))

        model = DLinearModel(input_chunk_length=128, output_chunk_length=28, batch_size=256, n_epochs=1000)
        model.fit(ts)
        
        future = pd.DataFrame()
        future['ds'] = pd.date_range(start='2023-03-04',periods=28,freq='D') #prophet_data['ID'].nunique() = 39
        forecast = model.predict(len(future['ds']))
        pred_y = forecast.values()
        pred_code = [str(code)] * len(pred_y)
        for y_val, id_val in zip(pred_y, pred_code):
            pred_list.append({'ID':id_val, 'y':y_val})

    pred = pd.DataFrame(pred_list)
    
    return pred

pred = ph_train(darts_data)

C:\Users\USER\anaconda3\envs\SU\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\USER\anaconda3\envs\SU\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Nam

Epoch 999: 100%|█████████████████████████████████████████████████████| 6/6 [00:00<00:00, 32.96it/s, train_loss=5.22e+6]

`Trainer.fit` stopped: `max_epochs=1000` reached.


Epoch 999: 100%|█████████████████████████████████████████████████████| 6/6 [00:00<00:00, 32.60it/s, train_loss=5.22e+6]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

`Trainer.fit` stopped: `max_epochs=1000` reached.


Epoch 999: 100%|██████████████████████████████████████████████████████| 6/6 [00:00<00:00, 44.60it/s, train_loss=3.8e+4]

GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.83it/s]


In [7]:
pred['y'] = pred['y'].explode().astype(float)
pred['y']

0       2898.126953
1       1169.042725
2       2607.216797
3       2760.611816
4       2561.528076
           ...     
1087     389.987061
1088     378.954102
1089     397.015167
1090     392.335022
1091     355.482300
Name: y, Length: 1092, dtype: float64

In [8]:
ss.loc[:,'answer'] = pred['y'].clip(0,)
ss.to_csv(os.path.join(sub_path,f"baseline_submission{sub_num}.csv"), index=False)